# HexMagic

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev import nbdev_export

In [ ]:
nbdev_export()

In [ ]:
#| hide
#import nbdev; nbdev.nbdev_export()
import sys
import math
sys.path.append(".")
sys.path.append("..")
from fastcore.basics import patch

#| export
## Introduction

The purpose of this library is to generate hex maps that are used in board games.

#| export
## Getting Started

In [ ]:
#| export
from HexMagic.primitives import Hex, MapCord 


In [ ]:
#| export

sampleHex = Hex(radius=50,center=MapCord(100,100))
sampleHex.vertices()

[(143.3,75.0),
 (143.3,125.0),
 (100.0,150.0),
 (56.7,125.0),
 (56.7,75.0),
 (100.0,50.0)]

In [ ]:
#| export
from HexMagic.styles import StyleCSS,  SVGBuilder

In [ ]:
#| export
#some drawing setup 
hexStyle = StyleCSS("HexStyle",fill="yellow",stroke="blue",stroke_width=2)
canvas = SVGBuilder()
canvas.width=200 ;canvas.height=200
canvas.add_style(hexStyle)

#add our hex to the canvas
#some drawing setup 


#add our hex to the canvas
sampleHex = Hex(radius=50,center=MapCord(100,100),style=hexStyle)
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()
canvas.adjust("main",sampleHex.svg())

#show our work
canvas.show()

#| export
Hex, short for hexagon, is a six-sided polygon that can be laid out in a honeycomb pattern that is space-efficient and strong. It also has a nicer perimeter than a square. This library groups them in a hexgrid, a honeycomb type structure

In [ ]:
#| export
from HexMagic.primitives import HexGrid

def sampleGrid(hexDim = 5, fill = "#ff7b00ff"):
    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)

    #Our new class
    grid = HexGrid.create_centered(hexDim,radius=30,style=hexStyle)

    #make sure we are ready to draw
    grid.update()
    return grid

sampleGrid().builder.show()

#| export
The `HexGrid` has a list of `Hex`, and we initialize them with a common style. The hexes are ordered starting from the top left.

In [ ]:
#| export

def sampleGrid(hexDim = 5, fill = "white",makeLabels = False):

    #drawing set up
    hexStyle = StyleCSS("HexStyle",fill=fill,stroke="blue",stroke_width=2)
    labelStyle = StyleCSS("labelStyle",fill=fill,stroke="black",stroke_width=1)
    
    grid = HexGrid.create_centered(hexDim,radius=40,style=hexStyle)

    #Need to add styles as we go along
    grid.builder.add_style(labelStyle)

    #itterate through the hexes
    for i in range(len(grid.hexes)):
        if makeLabels:
            grid.hexes[i].label = str(i) 
        grid.hexes[i].labelStyle = labelStyle.name

    grid.update()
    return grid
    

grid = sampleGrid(makeLabels=True)
print(len(grid.hexes)/2)
grid.builder.show()

12.5


#| export
### Absolute versus Realtive positions

This integer base system is great for absolute postioning, but is problematic from a realtive one. If we look at hex 12 two of its neighbors (11,13) are off by one. But the rest are going to be tricky since we need the number of rows and columns. Also even rows are different than odd ones.

What we need are directions.







In [ ]:
#| export
from HexMagic.primitives import HexPosition

grid = sampleGrid()

#mark the treasure
grid.hexes[12].label = "X"

#lets have a directions layer
arrowLayer = ""

#Use the realtive position class called HexPosition
for position in HexPosition.directions():
    i = grid.hexposition_to_index(position, 12)
    grid.hexes[i].label = position.label
    arrowLayer += grid.arrow(12,i)

grid.update()

#we can add a layer or update one in a builder using the adjust method
grid.builder.adjust("arrows",arrowLayer)

grid.builder.show()

In [ ]:
#| export
def showEast(middleIndex=12):
    grid = sampleGrid()

    #lets have a directions layer
    arrowLayer = ""
    
    #Mark our starting point
    grid.hexes[middleIndex].label = str(middleIndex)

    #Get a Direction
    eastDir = HexPosition.direction("E")

    #go part way
    i = grid.hexposition_to_index(eastDir, middleIndex)
    grid.hexes[i].label = "near"
    arrowLayer += grid.arrow(middleIndex,i)

    #keep going
    farEast = eastDir + eastDir
    j = grid.hexposition_to_index(farEast, middleIndex)
    grid.hexes[j].label = "far"
    arrowLayer += grid.arrow(i,j)

    grid.update()
    grid.builder.adjust("arrows",arrowLayer)
    return grid.builder.show()
showEast(12)


#| export
The nice part is that there is nothing special about where we start. It doesn't have to be the center

In [ ]:
#| export
showEast(6)

#| export
A common journey that you might want to do is to go out a certain distance and walk around in a ring.

In [ ]:
#| export
def showRing(middleIndex=12,ring=1):
    grid = sampleGrid()
    
    grid.hexes[middleIndex].label = "X"


    #lets have a directions layer
    arrowLayer = ""
    #Get a Direction
    eastDir = HexPosition.direction("E")

    #go out to the edge
    s = middleIndex
    for k in range(ring):
        i = grid.hexposition_to_index(eastDir, s)
        grid.hexes[i].label = f"r {k+1}"
        arrowLayer += grid.arrow(s,i)
        s = i

    origin = HexPosition.origin()

    #now lets go through the positions. by default it starts in the east like the sunrise.
    positions = origin.ring(ring)
    for pos in positions:
        i = grid.hexposition_to_index(pos,middleIndex ) 
        grid.hexes[i].label = f"r {pos.label}"
        if s != i:
            arrowLayer += grid.arrow(s,i)
        s = i

    grid.update()
    grid.builder.adjust("arrows",arrowLayer)
    return grid.builder.show()
    
showRing(12)

In [ ]:
#| export
showRing(12,ring=2)

#| export
### Coordinates

When representing hex directions, we need a coordinate system with several key properties:

1. **Additive**: Directions should combine naturally (e.g., two "east" moves = moving two hexes east)
2. **Reversible**: Every direction has a clear opposite (West is the reverse of East)
3. **Computationally efficient**: Operations should be fast and avoid floating-point errors

Since hexagons exist in the x,y plane, let's consider our options:

#### Cartesian Coordinates
The familiar x,y system works well for horizontal movements, but diagonal vectors involve factors of √3, requiring floating-point arithmetic and introducing potential rounding errors.

#### Polar Coordinates
Angles are clean multiples of 60°, but constant conversion between polar and Cartesian for rendering makes this impractical.

#### Dive a bit deeper
We have three pairs of directions (E/W,NW/SE,NE/SW) so we are going to need at 2 bits to repesent them. Lets build them

1. East can be (1, 0)
2. This makes West (-1, 0)
3. Let's make Southeast (0, 1)
4. Which makes Northwest (0, -1)
5. To get to Northeast using what we have built already. we go east (1,0), and then north west (0, -1) for (1,-1)
6. And that makes Southwest the opposite of Northeast (-1,1)

#### Cube Coordinates
What we have actually created are the first two digits of **cube coordinates** - a three-axis system (q, r, s) where:
- East: (1, 0, -1)
- West: (-1, 0, 1)
- Southeast: (0, 1, -1)
- Northwest: (0, -1, 1)
- Northeast: (1, -1, 0)
- Southwest: (-1, 1, 0)

The key constraint: **q + r + s = 0** always.

#### Why Cube Coordinates?

1. **Integer arithmetic only**: All operations use integers - no floating-point errors
2. **Natural addition**: Adding positions is simple component-wise addition
3. **Built-in validation**: The q + r + s = 0 constraint acts like double-entry accounting, catching errors immediately
4. **Distance calculation**: Manhattan distance in cube space equals hex distance: `(|q| + |r| + |s|) / 2`
5. **Rotation and reflection**: These operations become simple transformations of the three coordinates
6. **Line drawing**: Interpolating between hexes is straightforward with linear interpolation
7. **Range queries**: Finding all hexes within N steps is elegant: all positions where `(|q| + |r| + |s|) / 2 ≤ N`

The redundancy of the third coordinate isn't waste - it's a feature that makes many algorithms simpler and more robust.



In [ ]:
#| export
HexPosition.directions()

[HexPosition(-1, 1, 0, 'SW'),
 HexPosition(-1, 0, 1, 'W'),
 HexPosition(0, -1, 1, 'NW'),
 HexPosition(1, -1, 0, 'NE'),
 HexPosition(1, 0, -1, 'E'),
 HexPosition(0, 1, -1, 'SE')]

#| export
### Algorithms

The advantage of using the coordinates is that we can using existing algorithms for things like the path between points

In [ ]:
#| export
??HexPosition.line_to


```python
@patch
def line_to(self: HexPosition, other: HexPosition) -> list[HexPosition]:
    """Get line of hexes from self to other using linear interpolation"""
    n = self.distance(other)
    if n == 0:
        return [self]

    results = []
    for i in range(n + 1):
        t = i / n if n > 0 else 0
        # Lerp in cube coordinates
        q = self.q + (other.q - self.q) * t
        r = self.r + (other.r - self.r) * t
        s = self.s + (other.s - self.s) * t

        # Round to nearest hex
        results.append(HexPosition._cube_round(q, r, s))

    return results
```

**File:** `~/HexMagic/HexMagic/primitives.py`

In [ ]:
#| export
def line_demo(start_idx=20, end_idx=8):
    """Demo showing line drawing between two hexes using cube coordinates"""
    grid = sampleGrid()
    
    # Mark start and end
    grid.hexes[start_idx].label = "Start"
    grid.hexes[end_idx].label = "End"
    
    # Get the hex positions. This defaults to postion 0
    start_pos = grid.index_to_hexposition(start_idx)
    end_pos = grid.index_to_hexposition(end_idx)
    
    # Draw a line between them using cube coordinate interpolation
    path_positions = start_pos.line_to(end_pos)
    
    arrowLayer = ""
    prev_idx = start_idx
    
    for i, pos in enumerate(path_positions[1:], 1):  # Skip first (it's the start)
        curr_idx = grid.hexposition_to_index(pos)
        
        # Label intermediate hexes
        if curr_idx != end_idx and curr_idx >= 0:
            grid.hexes[curr_idx].label = str(i)
        
        # Draw arrow if valid index
        if curr_idx >= 0:
            arrowLayer += grid.arrow(prev_idx, curr_idx)
            prev_idx = curr_idx
    
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)
    return grid.builder.show()

line_demo(20, 8)


In [ ]:
#| export
def range_demo(center_idx=12, max_distance=2):
    """Demo showing all hexes within a certain distance"""
    grid = sampleGrid(hexDim=5, fill="lightgray")
    
    # Mark center
    grid.hexes[center_idx].label = "0"
    
    # Get center position
    center_pos = grid.index_to_hexposition(center_idx)
    
    # Color hexes by distance
    colors = [StyleCSS(x,fill=x,stroke="blue",stroke_width=2) for x in ["green","yellow", "orange", "red", "purple"]]
    for color in colors:
        grid.builder.add_style(color)
    
    for distance in range(1, max_distance + 1):
        # Get all positions at this distance
        positions = center_pos.ring(distance)
        
        for pos in positions:
            try:
                idx = grid.hexposition_to_index(pos, center_idx)
                if 0 <= idx < len(grid.hexes):
                    grid.hexes[idx].label = str(distance)
                    grid.hexes[idx].style = colors[distance-1]
                    # You could change fill color here if we modify the hex style
            except:
                pass  # Position outside grid
    
    grid.update()
    return grid.builder.show()

range_demo(12, 2)


In [ ]:
#| export
def rotation_demo(center_idx=12):
    """Demo showing rotation around a center hex"""
    grid = sampleGrid()
    
    grid.hexes[center_idx].label = "Center"
    
    # Start with a position to the east
    start_pos = HexPosition.direction("E")
    start_idx = grid.hexposition_to_index(start_pos, center_idx)
    grid.hexes[start_idx].label = "0°"
    
    arrowLayer = ""
    prev_idx = start_idx
    
    # Rotate around 60° at a time
    for i in range(1, 6):
        # Rotate the position
        rotated_pos = start_pos.rotate_right(i)
        curr_idx = grid.hexposition_to_index(rotated_pos, center_idx)
        
        grid.hexes[curr_idx].label = f"{i*60}°"
        arrowLayer += grid.arrow(prev_idx, curr_idx)
        prev_idx = curr_idx
    
    # Close the loop
    arrowLayer += grid.arrow(prev_idx, start_idx)
    
    grid.update()
    grid.builder.adjust("arrows", arrowLayer)
    return grid.builder.show()

rotation_demo(11)


#| export
This makes it very easy to check neighbors for coordinates, and there is an explicit function for neighbors in hexGrid

#| export
### Regions

We can combine both ideas of using integer indexes to access data and this ring system to check neighbors. A group of hex indexes can be combined into a set called hexregion.
Coming soon to you

In [ ]:
#| export
from HexMagic.primitives import HexRegion

In [ ]:
from dataclasses import dataclass